#  Test Binomial
Perform a fit to a set of data with binomial errors
like those derived from the division of two histograms.
Three different fits are performed and compared:

  -  simple least square fit to the divided histogram obtained
     from TH1::Divide with option b
  -  least square fit to the TGraphAsymmErrors obtained from
     TGraphAsymmErrors::BayesDivide
  -  likelihood fit performed on the dividing histograms using
     binomial statistics with the TBinomialEfficiency class

The first two methods are biased while the last one  is statistical correct.
Running the script passing an integer value n larger than 1, n fits are
performed and the bias are also shown.
To run the script :

 to show the bias performing 100 fits for 1000 events per "experiment"

```cpp
 root[0]: .x TestBinomial.C+
```

 to show the bias performing 100 fits for 1000 events per "experiment"

```cpp
          .x TestBinomial.C+(100, 1000)
```




**Author:** Rene Brun  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 01:33 AM.</small></i>

In [1]:
%%cpp -d
#include "TBinomialEfficiencyFitter.h"
#include "TVirtualFitter.h"
#include "TH1.h"
#include "TRandom3.h"
#include "TF1.h"
#include "TFitResult.h"
#include "TStyle.h"
#include "TCanvas.h"
#include "TLegend.h"
#include "TPaveStats.h"
#include "Math/IntegratorOptions.h"
#include <cassert>
#include <iostream>

 Arguments are defined. 

In [2]:
int nloop = 100;
int nevts = 100;
bool plot = false;
bool debug = false;
int seed = 111;

In [3]:
gStyle->SetMarkerStyle(20);
gStyle->SetLineWidth(2.0);
gStyle->SetOptStat(11);

TObjArray hbiasNorm;
hbiasNorm.Add(new TH1D("h0Norm", "Bias Histogram fit",100,-5,5));
hbiasNorm.Add(new TH1D("h1Norm","Bias Binomial fit",100,-5,5));
TObjArray hbiasThreshold;
hbiasThreshold.Add(new TH1D("h0Threshold", "Bias Histogram fit",100,-5,5));
hbiasThreshold.Add(new TH1D("h1Threshold","Bias Binomial fit",100,-5,5));
TObjArray hbiasWidth;
hbiasWidth.Add(new TH1D("h0Width", "Bias Histogram fit",100,-5,5));
hbiasWidth.Add(new TH1D("h1Width","Bias Binomial fit",100,-5,5));
TH1D* hChisquared = new TH1D("hChisquared",
   "#chi^{2} probability (Baker-Cousins)", 200, 0.0, 1.0);

TVirtualFitter::SetDefaultFitter("Minuit2");
ROOT::Math::IntegratorOneDimOptions::SetDefaultIntegrator("Gauss");

Note: in order to be able to use th1::fillrandom() to generate
       pseudo-experiments, we use a trick: generate "selected"
       and "non-selected" samples independently. These are
       statistically independent and therefore can be safely
       added to yield the "before selection" sample.

Define (arbitrarily?) a distribution of input events.
 Here: assume a x^(-2) distribution. Boundaries: [10, 100].

In [4]:
Double_t xmin =10, xmax = 100;
TH1D* hM2D = new TH1D("hM2D", "x^(-2) denominator distribution",
   45, xmin, xmax);
TH1D* hM2N = new TH1D("hM2N", "x^(-2) numerator distribution",
   45, xmin, xmax);
TH1D* hM2E = new TH1D("hM2E", "x^(-2) efficiency",
   45, xmin, xmax);

TF1*  fM2D = new TF1("fM2D", "(1-[0]/(1+exp(([1]-x)/[2])))/(x*x)",
   xmin, xmax);
TF1*  fM2N = new TF1("fM2N", "[0]/(1+exp(([1]-x)/[2]))/(x*x)",
   xmin, xmax);
TF1*  fM2Fit = new TF1("fM2Fit", "[0]/(1+exp(([1]-x)/[2]))",
   xmin, xmax);
TF1*  fM2Fit2 = 0;

TRandom3 rb(seed);

First try: use a single set of parameters.
 For each try, we need to find the overall normalization

In [5]:
Double_t normalization = 0.80;
Double_t threshold = 25.0;
Double_t width = 5.0;

fM2D->SetParameter(0, normalization);
fM2D->SetParameter(1, threshold);
fM2D->SetParameter(2, width);
fM2N->SetParameter(0, normalization);
fM2N->SetParameter(1, threshold);
fM2N->SetParameter(2, width);
Double_t integralN = fM2N->Integral(xmin, xmax);
Double_t integralD = fM2D->Integral(xmin, xmax);
Double_t fracN = integralN/(integralN+integralD);
Int_t nevtsN = rb.Binomial(nevts, fracN);
Int_t nevtsD = nevts - nevtsN;

std::cout << nevtsN << "  " << nevtsD << std::endl;

gStyle->SetOptFit(1111);

32  68


Generate many times to see the bias

In [6]:
for (int iloop = 0; iloop < nloop; ++iloop) {

  // generate pseudo-experiments
  hM2D->Reset();
  hM2N->Reset();
  hM2D->FillRandom(fM2D->GetName(), nevtsD);
  hM2N->FillRandom(fM2N->GetName(), nevtsN);
  hM2D->Add(hM2N);

  // construct the "efficiency" histogram
  hM2N->Sumw2();
  hM2E->Divide(hM2N, hM2D, 1, 1, "b");

  // Fit twice, using the same fit function.
  // In the first (standard) fit, initialize to (arbitrary) values.
  // In the second fit, use the results from the first fit (this
  // makes it easier for the fit -- but the purpose here is not to
  // show how easy or difficult it is to obtain results, but whether
  // the CORRECT results are obtained or not!).

  fM2Fit->SetParameter(0, 0.5);
  fM2Fit->SetParameter(1, 15.0);
  fM2Fit->SetParameter(2, 2.0);
  fM2Fit->SetParError(0, 0.1);
  fM2Fit->SetParError(1, 1.0);
  fM2Fit->SetParError(2, 0.2);
  TH1 * hf = fM2Fit->GetHistogram();
  // std::cout << "Function values " << std::endl;
  // for (int i = 1; i <= hf->GetNbinsX(); ++i)
  //    std::cout << hf->GetBinContent(i) << "  ";
  // std::cout << std::endl;

  TCanvas* cEvt;
  if (plot) {
    cEvt = new TCanvas(Form("cEnv%d",iloop),
                       Form("plots for experiment %d", iloop),
                       1000, 600);
    cEvt->Divide(1,2);
    cEvt->cd(1);
    hM2D->DrawCopy("HIST");
    hM2N->SetLineColor(kRed);
    hM2N->DrawCopy("HIST SAME");
    cEvt->cd(2);
  }
  for (int fit = 0; fit < 2; ++fit) {
    Int_t status = 0;
    switch (fit) {
    case 0:
    {
       // TVirtualPad * pad = gPad;
       // new TCanvas();
       // fM2Fit->Draw();
       // gPad = pad;
       TString optFit = "RN";
       if (debug) optFit += TString("SV");
       TFitResultPtr res = hM2E->Fit(fM2Fit, optFit);
       if (plot) {
          hM2E->DrawCopy("E");
          fM2Fit->SetLineColor(kBlue);
          fM2Fit->DrawCopy("SAME");
       }
       if (debug) res->Print();
       status = res;
       break;
    }
    case 1:
    {
       // if (fM2Fit2) delete fM2Fit2;
       // fM2Fit2 = dynamic_cast<TF1*>(fM2Fit->Clone("fM2Fit2"));
       fM2Fit2 = fM2Fit; // do not clone/copy the function
       if (fM2Fit2->GetParameter(0) >= 1.0)
       fM2Fit2->SetParameter(0, 0.95);
       fM2Fit2->SetParLimits(0, 0.0, 1.0);

       // TVirtualPad * pad = gPad;
       // new TCanvas();
       // fM2Fit2->Draw();
       // gPad = pad;

       TBinomialEfficiencyFitter bef(hM2N, hM2D);
       TString optFit = "RI S";
       if (debug) optFit += TString("V");
       TFitResultPtr res = bef.Fit(fM2Fit2,optFit);
       status = res;
       if (status !=0) {
          std::cerr << "Error performing binomial efficiency fit, result = "
          << status << std::endl;
          res->Print();
          continue;
       }
       if (plot) {
          fM2Fit2->SetLineColor(kRed);
          fM2Fit2->DrawCopy("SAME");
       
          bool confint = (status == 0);
          if (confint) {
             // compute confidence interval on fitted function
             auto htemp = fM2Fit2->GetHistogram();
             ROOT::Fit::BinData xdata;
             ROOT::Fit::FillData(xdata, fM2Fit2->GetHistogram() );
             TGraphErrors gr(fM2Fit2->GetHistogram() );
             res->GetConfidenceIntervals(xdata, gr.GetEY(), 0.68, false);
             gr.SetFillColor(6);
             gr.SetFillStyle(3005);
             gr.DrawClone("4 same");
          }
       }
       if (debug) {
          res->Print();
       }
    }
    }

    if (status != 0) break;

    Double_t fnorm = fM2Fit->GetParameter(0);
    Double_t enorm = fM2Fit->GetParError(0);
    Double_t fthreshold = fM2Fit->GetParameter(1);
    Double_t ethreshold = fM2Fit->GetParError(1);
    Double_t fwidth = fM2Fit->GetParameter(2);
    Double_t ewidth = fM2Fit->GetParError(2);
    if (fit == 1) {
       fnorm = fM2Fit2->GetParameter(0);
       enorm = fM2Fit2->GetParError(0);
       fthreshold = fM2Fit2->GetParameter(1);
       ethreshold = fM2Fit2->GetParError(1);
       fwidth = fM2Fit2->GetParameter(2);
       ewidth = fM2Fit2->GetParError(2);
       hChisquared->Fill(fM2Fit2->GetProb());
    }

    TH1D* h = dynamic_cast<TH1D*>(hbiasNorm[fit]);
    h->Fill((fnorm-normalization)/enorm);
    h = dynamic_cast<TH1D*>(hbiasThreshold[fit]);
    h->Fill((fthreshold-threshold)/ethreshold);
    h = dynamic_cast<TH1D*>(hbiasWidth[fit]);
    h->Fill((fwidth-width)/ewidth);
  }
}


TCanvas* c1 = new TCanvas("c1",
   "Efficiency fit biases",10,10,1000,800);
c1->Divide(2,2);

TH1D *h0, *h1;
c1->cd(1);
h0 = dynamic_cast<TH1D*>(hbiasNorm[0]);
h0->Draw("HIST");
h1 = dynamic_cast<TH1D*>(hbiasNorm[1]);
h1->SetLineColor(kRed);
h1->Draw("HIST SAMES");
TLegend* l1 = new TLegend(0.1, 0.75, 0.5, 0.9,
   "plateau parameter", "ndc");
l1->AddEntry(h0, Form("histogram: mean = %4.2f RMS = \
   %4.2f", h0->GetMean(), h0->GetRMS()), "l");
l1->AddEntry(h1, Form("binomial : mean = %4.2f RMS = \
   %4.2f", h1->GetMean(), h1->GetRMS()), "l");
l1->Draw();

c1->cd(2);
h0 = dynamic_cast<TH1D*>(hbiasThreshold[0]);
h0->Draw("HIST");
h1 = dynamic_cast<TH1D*>(hbiasThreshold[1]);
h1->SetLineColor(kRed);
h1->Draw("HIST SAMES");
TLegend* l2 = new TLegend(0.1, 0.75, 0.5, 0.9,
   "threshold parameter", "ndc");
l2->AddEntry(h0, Form("histogram: mean = %4.2f RMS = \
   %4.2f", h0->GetMean(), h0->GetRMS()), "l");
l2->AddEntry(h1, Form("binomial : mean = %4.2f RMS = \
   %4.2f", h1->GetMean(), h1->GetRMS()), "l");
l2->Draw();

c1->cd(3);
h0 = dynamic_cast<TH1D*>(hbiasWidth[0]);
h0->Draw("HIST");
h1 = dynamic_cast<TH1D*>(hbiasWidth[1]);
h1->SetLineColor(kRed);
h1->Draw("HIST SAMES");
TLegend* l3 = new TLegend(0.1, 0.75, 0.5, 0.9, "width parameter", "ndc");
l3->AddEntry(h0, Form("histogram: mean = %4.2f RMS = \
   %4.2f", h0->GetMean(), h0->GetRMS()), "l");
l3->AddEntry(h1, Form("binomial : mean = %4.2f RMS = \
   %4.2f", h1->GetMean(), h1->GetRMS()), "l");
l3->Draw();

c1->cd(4);
hChisquared->Draw("HIST");


****************************************
Minimizer is Minuit2
Chi2                      =     0.350714
NDf                       =            3
Edm                       =  1.27001e-06
NCalls                    =           81
p0                        =     0.694132   +/-   0.210029    
p1                        =      19.3471   +/-   5.85483     
p2                        =       5.2245   +/-   5.11013     

****************************************
Minimizer is Minuit2
Chi2                      =      1.64738
NDf                       =            5
Edm                       =  1.01171e-06
NCalls                    =          137
p0                        =     0.677329   +/-   0.135567     	 (limited)
p1                        =      15.5028   +/-   4.11118     
p2                        =      4.10441   +/-   2.77044     

****************************************
Minimizer is Minuit2
Chi2                      =     0.928456
NDf                       =            6
Edm              

Info in <TBinomialEfficiencyFitter::result>:  chi2 28.616156 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 38.726065 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 23.428731 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 25.436918 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 18.112302 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 31.946587 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 35.614024 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights stru


****************************************
Minimizer is Minuit2
Chi2                      =      2.39556
NDf                       =            8
Edm                       =  4.51695e-08
NCalls                    =           81
p0                        =     0.564008   +/-   0.188334     	 (limited)
p1                        =      19.2305   +/-   5.64705     
p2                        =      4.00431   +/-   3.2294      

****************************************
Minimizer is Minuit2
Chi2                      =      2.71864
NDf                       =            3
Edm                       =  2.02651e-07
NCalls                    =           84
p0                        =     0.430955   +/-   0.103386     	 (limited)
p1                        =      15.3556   +/-   1.58614     
p2                        =      1.18646   +/-   1.05279     

****************************************
Minimizer is Minuit2
Chi2                      =      3.91497
NDf                       =            3
Edm  

Info in <TBinomialEfficiencyFitter::result>:  chi2 33.160428 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 28.884040 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 27.548549 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 24.338757 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 14.294550 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 27.886514 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TBinomialEfficiencyFitter::Fit>: Abnormal termination of minimization.
Info in <TBinomialEfficiencyFitter::resul


****************************************
         Invalid FitResult  (status = 4 )
****************************************
Minimizer is Minuit2
MinFCN                    =     -5.64605
NDf                       =            0
Edm                       =  0.000382221
NCalls                    =         1345
Par_0                     =      9823.02   +/-   10056.8     
Par_1                     =      146.017   +/-   23.1477     
Par_2                     =      10.7238   +/-   1.67966     

****************************************
Minimizer is Minuit2
Chi2                      =      3.05436
NDf                       =            5
Edm                       =  1.66356e-05
NCalls                    =          297
p0                        =     0.999547   +/-   0.735933     	 (limited)
p1                        =      47.3138   +/-   36.6961     
p2                        =      43.5419   +/-   61.6394     

****************************************
         Invalid FitResult  (status =

...

Info in <TBinomialEfficiencyFitter::result>:  chi2 19.975527 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TBinomialEfficiencyFitter::Fit>: Abnormal termination of minimization.
Info in <TBinomialEfficiencyFitter::result>:  chi2 -13.822347 ndf 0 
Error performing binomial efficiency fit, result = 4
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 22.974282 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 28.004052 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 20.312990 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 22.326995 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure alr


****************************************
         Invalid FitResult  (status = 4 )
****************************************
Minimizer is Minuit2
MinFCN                    =     -7.86545
NDf                       =            0
Edm                       =  0.000208126
NCalls                    =         1347
Par_0                     =      24213.5   +/-   41584.5     
Par_1                     =      150.325   +/-   26.5019     
Par_2                     =      10.3799   +/-   1.6973      

****************************************
Minimizer is Minuit2
Chi2                      =      2.75509
NDf                       =            9
Edm                       =  9.98946e-08
NCalls                    =           69
p0                        =     0.593401   +/-   0.101469     	 (limited)
p1                        =      18.8032   +/-   2.57519     
p2                        =      2.53183   +/-   1.7654      

****************************************
Minimizer is Minuit2
Chi2            

Warning in <TBinomialEfficiencyFitter::Fit>: Abnormal termination of minimization.
Info in <TBinomialEfficiencyFitter::result>:  chi2 -15.730894 ndf 0 
Error performing binomial efficiency fit, result = 4
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 17.653827 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 26.186074 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 21.604809 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 32.763550 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 30.722601 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure alr


****************************************
         Invalid FitResult  (status = 4 )
****************************************
Minimizer is Minuit2
MinFCN                    =      8.96453
NDf                       =            0
Edm                       =  2.84297e-05
NCalls                    =         1753
Par_0                     =      7893.33   +/-   24437.7     
Par_1                     =      242.747   +/-   70.5949     
Par_2                     =      20.7763   +/-   2.9814      

****************************************
Minimizer is Minuit2
Chi2                      =      4.29305
NDf                       =            9
Edm                       =  3.06585e-06
NCalls                    =           82
p0                        =     0.687721   +/-   0.16847      	 (limited)
p1                        =      22.2779   +/-   3.60769     
p2                        =      4.13806   +/-   1.75728     

****************************************
Minimizer is Minuit2
Chi2            

Warning in <TBinomialEfficiencyFitter::Fit>: Abnormal termination of minimization.
Info in <TBinomialEfficiencyFitter::result>:  chi2 17.929067 ndf 0 
Error performing binomial efficiency fit, result = 4
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 13.032291 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 42.771081 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 41.549220 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 26.626798 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TBinomialEfficiencyFitter::result>:  chi2 37.223499 ndf 0 
Warning in <TH1D::Sumw2>: Sum of squares of weights structure alre

Draw all canvases 

In [7]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()